# 03 · Flower Simulation (placeholder)

This notebook will host a multi-node Flower simulation for federated learning.

TODO (you): Provide the dataset partitioning strategy, client configs, and server strategy details so we can complete this notebook.
Notes: 
- We plan to support CPU/GPU mixed clients.
- Metrics should be logged to CSV and compared later in 04.
- Provide any required credentials or cluster details if needed (none stored in repo).